# Coursera Capstone
This notebook will be used for the IBM Data Science capstone project.

##Imports

In [0]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

## Segmenting and Clustering Neighborhoods in Toronto

In [317]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(URL).text
soup = BeautifulSoup(html, 'lxml')
table = soup.find('table', attrs={'class':'wikitable sortable'})

new_table = pd.DataFrame(columns=range(0,3), index = [0])
table_body = table.find('tbody')

data = []
rows = table_body.find_all('tr')
for row in rows:
  cols = row.find_all('td')
  row = [row.text for row in cols]
  # Ignore cells with a borough that is Not assigned.
  if "Not assigned" not in row:
    data.append(row)

df = pd.DataFrame(data, columns=['Postcode', 'Borough', 'Neighborhood_'])
df = df.replace(r'\n',' ', regex=True) 
df.head(10)

,Postcode,Borough,Neighborhood_
0,None,None,None
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Downtown Toronto,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


More than one neighborhood can exist in one postal code area. For example, in the table above, you will notice that M1B is listed twice and has two neighborhoods: Rouge and Malvern. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [318]:
df_neigh = df.groupby('Postcode')['Neighborhood_'].agg({'Neighborhood_':'first', 'Neighborhood':', '.join})
df = pd.merge(df, df_neigh, how='inner', on=['Neighborhood_'] )
df.drop('Neighborhood_', axis=1, inplace=True)
df.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights , Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge , Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens , Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson , Garden District"


In [320]:
df.shape

(105, 3)